# Tree Methods Consulting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('dog').getOrCreate()

In [6]:
data = spark.read.csv('/FileStore/tables/dog_food.csv', inferSchema=True, header=True)

In [7]:
#Tamaño del dataset
print(data.count(), len(data.columns))

490 5

In [8]:
data.toPandas().head()

,A,B,C,D,Spoiled
0,4,2,12.0,3,1.0
1,5,6,12.0,7,1.0
2,6,2,13.0,6,1.0
3,4,2,12.0,1,1.0
4,4,2,12.0,3,1.0


In [9]:
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

Aprovecharemos el atributo "feature importance" que el método random forest trae para determinar qué variable(s) tiene más efecto en la mezcla química.

In [11]:
#Convertimos al formato requerido por pyspark
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

Out[9]: ['A', 'B', 'C', 'D', 'Spoiled']

In [13]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features','Spoiled')

In [16]:
from pyspark.ml.classification import RandomForestClassifier

In [17]:
#No dividiré en train y test set ya que el objetivo es encontrar la feature con más efecto y no generar un modelo propiamente tal

rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features', numTrees=100)

In [18]:
rfc_model = rfc.fit(final_data)

In [19]:
rfc_model.featureImportances

Out[23]: SparseVector(4, {0: 0.0202, 1: 0.0201, 2: 0.9367, 3: 0.023})

De acuerdo a lo que devuelve el método featureImportances vemos que el preservante C es el que tiene más efecto en la mezcla y puede estar echándola a perder.